In [1]:
import imprint as ip
ip.setup_nb()

[worker_id=None] 2023-03-16 19:50:37,677 - imprint - DEBUG 
Enabling 64-bit floats in JAX.


In [2]:
from confirm.cloud.coiled_backend import CoiledBackend, setup_cluster
import confirm.adagrid as ada
from imprint.models.ztest import ZTest1D

cluster = setup_cluster(idle_timeout="2 hours")

[worker_id=None] 2023-03-16 19:50:38,740 - confirm.cloud.coiled_backend - DEBUG 
Coiled environment:
 
name: confirm
channels:
  - conda-forge
  - nvidia
dependencies:
  - python=3.10
  - nvidia:cuda-toolkit=11.8
  - conda-forge:cudnn=8.4
  - pip
  - pip:
    - absl-py==1.4.0
    - cloudpickle==2.0.0
    - colorama==0.4.6
    - decorator==5.1.1
    - dm-tree==0.1.8
    - duckdb==0.7.1
    - gast==0.5.3
    - mpmath==1.3.0
    - multipledispatch==0.6.0
    - numpy==1.24.2
    - numpyro==0.11.0
    - opt-einsum==3.3.0
    - pandas==1.5.3
    - python-dateutil==2.8.2
    - pytz==2022.7.1
    - scipy==1.10.1
    - six==1.16.0
    - sympy==1.11.1
    - synchronicity==0.2.14
    - tensorflow-probability==0.19.0
    - tqdm==4.65.0
    - --find-links https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
    - jax[cuda11_cudnn82]==0.4.5
    - dask[complete]==2023.3.0
    - pynvml==11.5.0

[worker_id=None] 2023-03-16 19:50:38,742 - confirm.cloud.coiled_backend - DEBUG 
Creating soft

Output()

/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/distributed/client.py:1361: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.3.2  | None      | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [32]:
backend = CoiledBackend(cluster=cluster)
g = ip.cartesian_grid(theta_min=[-1], theta_max=[1], null_hypos=[ip.hypo("x0 < 0")])
db = ada.ada_calibrate(
    ZTest1D,
    g=g,
    nB=5,
    prod=True,
    n_zones=1,
    backend=CoiledBackend(cluster=cluster),
)


[worker_id=None] 2023-03-16 19:57:53,833 - imprint.grid - DEBUG 
_gen_short_uuids(n=2, worker_id=1, t=1679011073, n_bits=18, worker_bits=18) = [4700297928252850176 4700297928252850177, ...]:
[worker_id=1] 2023-03-16 19:57:55,515 - confirm.adagrid.init - DEBUG 
Initialized database with 1 tiles and 1 null hypos. The tiles are split between 1 zones with packet_size=1024.
[worker_id=1] 2023-03-16 19:57:55,517 - confirm.adagrid.init - INFO 
Config (minus system info): 
{'model_seed': 0, 'alpha': 0.025, 'init_K': 8192, 'n_K_double': 4, 'bootstrap_seed': 0, 'nB': 5, 'tile_batch_size': None, 'grid_target': 0.001, 'bias_target': 0.001, 'std_target': 0.002, 'calibration_min_idx': 40, 'n_steps': 100, 'timeout': 43200, 'step_size': 1024, 'packet_size': 1024, 'coordinate_every': 1, 'n_zones': 1, 'prod': True, 'job_name': None, 'model_name': 'ZTest1D', 'model_kwargs_json': '{}', 'worker_id': 1, 'jax_platform': 'cpu', 'git_hash': 'fe878e935260206268346f7dea2fc8d6e3cb4f75', 'platform': 'macOS-13.2.1-

In [37]:
import numpy as np
d = 3
g = ip.cartesian_grid(
    theta_min=np.full(d, -1),
    theta_max=np.full(d, 0),
    null_hypos=[ip.hypo("theta0 > theta1")],
)
db = ada.ada_validate(
    ZTest1D,
    g=g,
    lam=-1.96,
    prod=False,
    n_K_double=7,
    max_target=0.001,
    global_target=0.002,
    step_size=2**15,
    packet_size=2**12,
    n_steps=10,
    backend=CoiledBackend(cluster=cluster)
    # backend=ModalBackend(n_workers=1, gpu="any"),
)

[worker_id=None] 2023-03-16 19:59:41,084 - imprint.grid - DEBUG 
_gen_short_uuids(n=8, worker_id=1, t=1679011181, n_bits=18, worker_bits=18) = [4700305349956337664 4700305349956337665 4700305349956337666, ...]:
[worker_id=None] 2023-03-16 19:59:41,086 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 3. Padding with zeros.
[worker_id=None] 2023-03-16 19:59:41,089 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 3. Padding with zeros.
[worker_id=None] 2023-03-16 19:59:41,091 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 3. Padding with zeros.
[worker_id=None] 2023-03-16 19:59:41,093 - imprint.planar_null - DEBUG 
HyperPlane has dimension 2 but grid has dimension 3. Padding with zeros.
[worker_id=None] 2023-03-16 19:59:41,097 - imprint.grid - DEBUG 
_gen_short_uuids(n=8, worker_id=1, t=1679011182, n_bits=18, worker_bits=18) = [4700305418675814400 4700305418675814401 4700305418675814402, ...]

In [30]:
tdf = db.con.query('select * from tiles where step_id=4 and packet_id=1').df()
tdf = tdf[tdf['K'] < 30000]
tdf.shape

(816, 17)

In [5]:
import confirm.cloud.coiled_backend as coiled_backend
client = await cluster.get_client()
coiled_backend.reset_confirm_imprint(client)

In [22]:
from confirm.adagrid.validate import AdaValidate
worker_args_fut = client.scatter((
    ZTest1D,
    (0, 2**14),
    dict(),
    AdaValidate,
    dict(
        lam=-1.96,
        tile_batch_size=64,
        delta=0.01,
        global_target = 0.001, 
        max_target = 0.001, 
        init_K=2**12,
        n_K_double=4,
    ),
), broadcast=True)

In [31]:
client.submit(coiled_backend.dask_process_tiles, worker_args_fut, tdf).result()

(                      id  active            parent_id   theta0   theta1  \
 0    4700269340950528208    True  4700269203511574948 -0.03125 -0.21875   
 1    4700269340950528209    True  4700269203511574949 -0.03125 -0.21875   
 2    4700269340950528210    True  4700269203511574950 -0.03125 -0.15625   
 3    4700269340950528211    True  4700269203511574951 -0.03125 -0.15625   
 4    4700269340950528212    True  4700269203511574956 -0.03125 -0.21875   
 ..                   ...     ...                  ...      ...      ...   
 827  4700269478389481782    True  4700269340950528859 -0.21875 -0.21875   
 828  4700269478389481784    True  4700269340950528924 -0.21875 -0.21875   
 829  4700269478389481786    True  4700269340950528925 -0.21875 -0.21875   
 830  4700269478389481788    True  4700269340950528926 -0.21875 -0.21875   
 831  4700269478389481790    True  4700269340950528927 -0.21875 -0.21875   
 
       theta2   radii0   radii1   radii2  null_truth0  coordination_id  \
 0   -0.4687